```{contents}
```
### Multi-Tool Agents 


A **multi-tool agent** is an agentic LLM system that can **choose among multiple tools**, **call them in sequence or combination**, and **use their outputs together** to produce a final answer.

> Instead of a single function call, the agent dynamically orchestrates **several tools** to solve a task.

---

### Why Multi-Tool Agents Are Needed

Single-tool usage fails when:

* Information is spread across systems
* Tasks require multiple steps
* Decisions depend on intermediate results

Multi-tool agents enable:

* End-to-end workflows
* Cross-system reasoning
* Higher accuracy for complex queries

---

### Conceptual Flow

```
User Query
   ↓
LLM Reasoning
   ↓
Tool Selection (Tool A)
   ↓
Observation
   ↓
LLM Reasoning
   ↓
Tool Selection (Tool B)
   ↓
Observation
   ↓
Final Answer
```

This loop may run **multiple times**.

---

### Multi-Tool Agent vs Single-Tool Agent

| Aspect            | Single-Tool | Multi-Tool |
| ----------------- | ----------- | ---------- |
| Tools available   | 1           | Many       |
| Decision logic    | Simple      | Dynamic    |
| Steps             | One         | Many       |
| Complex workflows | ❌           | ✅          |

---

### Common Tool Types in Multi-Tool Agents

#### Retrieval Tools

* Vector search
* Hybrid search
* Knowledge base lookup

#### Data Tools

* Database queries
* CSV / log readers

#### Action Tools

* Create/update tickets
* Trigger workflows

#### Utility Tools

* Calculator
* Date/time logic
* Validators

---

### Typical Multi-Tool Use Case

**User:**
“Are Jira tickets exceeding SLA today, and notify the on-call engineer if yes.”

**Agent reasoning (simplified):**

```
Thought: Need ticket count → use ticket_count tool
Observation: 128

Thought: Need SLA threshold → use sla_lookup tool
Observation: 100

Thought: 128 > 100 → notify engineer
Action: send_notification
Observation: success

Final Answer: Yes, tickets exceed SLA. Notification sent.
```

---

### Multi-Tool Agents in LangChain

LangChain supports multi-tool agents via:

* **ReAct agents**
* **OpenAI Tools agents**
* **LangGraph (recommended for production)**

All tools are registered upfront.

---

### Minimal Demonstration (LangChain – Conceptual)

#### Step 1: Define Multiple Tools



In [1]:
from langchain_classic.tools import tool

@tool
def ticket_count(source: str) -> int:
    """Return number of tickets"""
    return 128

@tool
def sla_threshold() -> int:
    """Return SLA ticket threshold"""
    return 100



---

#### Step 2: Create the Chat Model



In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)




---

#### Step 3: Create the Agent



In [6]:
from langchain_classic.agents import create_react_agent
from langchain_classic import hub

# Pull the ReAct prompt template from LangChain Hub
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm=llm,
    tools=[ticket_count, sla_threshold],
    prompt=prompt
)



---

#### Step 4: Execute the Agent



In [7]:
from langchain_classic.agents import AgentExecutor

executor = AgentExecutor(
    agent=agent,
    tools=[ticket_count, sla_threshold],
    verbose=True
)

executor.invoke({
    "input": "Are Jira tickets exceeding SLA?"
})




> Entering new AgentExecutor chain...
To determine if Jira tickets are exceeding the SLA, I need to compare the number of tickets with the SLA threshold. First, I will retrieve the number of tickets. 

Action: ticket_count  
Action Input: "Jira"  128I have the number of Jira tickets, which is 128. Now, I need to retrieve the SLA threshold to compare it with the number of tickets.

Action: sla_threshold  
Action Input: None  100I now have both the number of Jira tickets (128) and the SLA threshold (100). Now I can compare them to determine if the tickets are exceeding the SLA.

Final Answer: Yes, Jira tickets are exceeding SLA since 128 > 100.

> Finished chain.


{'input': 'Are Jira tickets exceeding SLA?',
 'output': 'Yes, Jira tickets are exceeding SLA since 128 > 100.'}



The agent decides:

* Which tool to call
* In what order
* How to combine results

---

### How Tool Selection Works

The LLM chooses tools based on:

* Tool **name**
* Tool **description**
* Tool **input schema**
* Current reasoning state

Clear tool descriptions are critical.

---

### Multi-Tool Agents vs Sequential Chains

| Aspect       | Sequential Chain | Multi-Tool Agent |
| ------------ | ---------------- | ---------------- |
| Flow         | Fixed            | Dynamic          |
| Branches     | ❌                | ✅                |
| Tool choice  | Predefined       | LLM-decided      |
| Adaptability | Low              | High             |

---

### Multi-Tool Agents vs LangGraph

| Aspect            | Multi-Tool Agent | LangGraph              |
| ----------------- | ---------------- | ---------------------- |
| Control           | LLM-driven       | Explicit state machine |
| Predictability    | Medium           | High                   |
| Production safety | Medium           | Very High              |

**Rule:**

* Use agents for flexibility
* Use LangGraph for mission-critical workflows

---

### Guardrails for Multi-Tool Agents (Critical)

* Tool allowlist
* Read vs write tool separation
* Input validation
* Max iteration limits
* Low temperature
* Human approval for destructive actions

---

### Common Failure Modes

#### Tool confusion

* Poor descriptions
* Overlapping tool purposes

#### Infinite loops

* No stop condition

#### Tool overuse

* Calling tools unnecessarily

---

### Best Practices

* Keep tools small and single-purpose
* Write precise tool descriptions
* Limit number of exposed tools
* Log every tool call
* Add fallback logic

---

### Production Use Cases

* IT support automation
* Incident management
* Customer support bots
* DevOps copilots
* Data analysis assistants

---

### Interview-Ready Summary

> “Multi-tool agents allow an LLM to dynamically choose and orchestrate multiple tools in a reasoning loop. They enable complex, multi-step workflows but require strong guardrails for production use.”

---

### Rule of Thumb

* **One deterministic action → Tool calling**
* **Multiple actions → Multi-tool agent**
* **High safety required → LangGraph**
* **Enterprise automation → Multi-tool + guardrails**